In [ ]:

import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [ ]:
#use folders of audio image data

#Training data
from keras.preprocessing.image import ImageDataGenerator

image_datagen = ImageDataGenerator(rescale=1./255)

image_generator_train = image_datagen.flow_from_directory(
    '/content/drive/MyDrive/phoebe/phoebe_chunk_spec/',
    target_size=(150, 150),
    batch_size=4,
    class_mode='binary')

#Validation data
image_generator_val = image_datagen.flow_from_directory(
    '/content/drive/MyDrive/phoebe/phoebe_chunk_val_spec/',
    target_size=(150, 150),
    batch_size=4,
    class_mode='binary')



In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, kernel_size = (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    # Only 1 output neuron.
    tf.keras.layers.Dense(1, activation='sigmoid')  
])
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(image_generator_train,
                    validation_data = (image_generator),
                    steps_per_epoch=10,
                    epochs=100,
                    verbose=2)


save model

In [ ]:
model.save('/content/drive/MyDrive/phoebe/trigger_word_model.h5')

Load model and test it on an audio image, transformed into an array of numbers

In [ ]:
import tensorflow as tf
new_model = tf.keras.models.load_model('/content/drive/MyDrive/phoebe/trigger_word_model.h5')

In [ ]:
#use 1 validation audio for testing
from keras.preprocessing import image
path = '/content/drive/MyDrive/phoebe/phoebe_chunk_val_spec/class1/audio00.png'
img=image.load_img(path, target_size=(150, 150))

x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
#images = np.vstack([x])

print(new_model.predict(x))